In [1]:
import os
import re
import itertools
import pickle
from collections import Counter,defaultdict
try:
	import pandas as pd
	import numpy as np
	from nltk.corpus import stopwords
	from nltk.stem import PorterStemmer
	import gensim
	from gensim.models import Word2Vec
	from nltk.stem import WordNetLemmatizer
	from keras.utils import np_utils
except:
	print("require modules: keras,gensim,nltk.stem,nltk.corpus,nltk.stem, please install it.")
	exit()

Using TensorFlow backend.


In [2]:
reg1 = re.compile("\.txt$")
reg2 = re.compile("([0-9]+)\.txt")
reg3 = re.compile(".*_([0-9])\.txt")
reg4 = re.compile("\[.+\]")
reg5 = re.compile("info\.txt")
lyrics_dic = {}
#iter all directory and load all song(txt file)
for i in os.listdir():
    if os.path.isdir(i):
        for path,sub,items in os.walk(i):
            if any([reg1.findall(item) for item in items]):
                for item in items:
                    if reg5.findall(item):
                        continue
                    if reg3.findall(item):
                        num = ["0"+reg3.findall(item)[0]]
                        name = "_".join(path.split("/") + num)
                    else:
                        name = "_".join(path.split("/") + reg2.findall(item))

                    with open(os.path.join(path,item),"r",encoding="utf8", errors='ignore') as f:
                        lyrics = "".join(f.readlines())
                        lyrics = reg4.subn("",lyrics)[0]
                        lyrics_dic[name] = lyrics

In [3]:
len(lyrics_dic.keys())

777

In [4]:
df=pd.DataFrame(columns=["lyrics","mood"])

In [5]:
for key in lyrics_dic.keys():
    #print(key)
    if "Happy" in key:
        
        df=df.append({"lyrics":lyrics_dic[key],"mood":"1"},ignore_index=True)
    elif "Sad" in key:
        df=df.append({"lyrics":lyrics_dic[key],"mood":"2"},ignore_index=True)
    elif "Angry" in key:
        df=df.append({"lyrics":lyrics_dic[key],"mood":"3"},ignore_index=True)
    elif "Relaxed" in key:
        df=df.append({"lyrics":lyrics_dic[key],"mood":"4"},ignore_index=True)

In [6]:
df.head()

,lyrics,mood
0,Put your lips close to mine\nAs long as they d...,1
1,"My lullaby,hung out to dry\nWhat's up with tha...",1
2,Though you've played at love and lost\nAnd sor...,1
3,we know we are the ones\nwho do it numb again\...,1
4,Another day has come and gone\nThey fade away ...,1


In [7]:
df = df.sample(frac=1).reset_index(drop=True)

In [81]:
df_new=pd.read_csv("df_mood_backup.csv")

In [128]:
df_new.tail()

,lyrics,mood
600,You should have listened to what mama said\nAn...,1
601,Spoken Intro:\nYou ever want something\nThat y...,2
602,All the world's a masquerade\nMade up of fools...,2
603,Chains of despair cloacked by darkness\nThe th...,3
604,One more night\nOne more night\n\nI've been tr...,2


In [83]:
df_new=df_new[["lyrics","mood"]]

In [84]:
df3=df.append(df_new,ignore_index=True)

In [85]:
df3["mood"]=df3["mood"].apply(str)

In [91]:
df3.shape

(1382, 2)

In [93]:
set(df3["mood"].values)
df=df3

In [107]:
data_X = df['lyrics'].values
data_Y = df['mood'].values
import nltk
import string
import re

porter_stemmer = nltk.stem.porter.PorterStemmer()
def porter_tokenizer(text, stemmer=porter_stemmer):
    lower_txt = text.lower()
    tokens = nltk.wordpunct_tokenize(lower_txt)
    stems = [porter_stemmer.stem(t) for t in tokens]
    no_punct = [s for s in stems if re.match('^[a-zA-Z]+$', s) is not None]
    return no_punct
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
            encoding='utf-8',
            decode_error='replace',
            strip_accents='unicode',
            analyzer='word',
            binary=False,
            stop_words="english",
            tokenizer=porter_tokenizer
    )

In [108]:



data_X = tfidf.fit_transform(data_X)




In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.4, random_state=0)

In [122]:

from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
classifier_rbf = LinearSVC(multi_class="ovr")
classifier_rbf.fit(X_train, y_train)
prediction_rbf = classifier_rbf.predict(X_test)
from sklearn.metrics import classification_report,accuracy_score
def prediction_evaluation(model_name,model,test_X,test_Y):
    y_pred = model.predict(test_X)
    cm = metrics.confusion_matrix(test_Y, y_pred)
    print(cm)
    print(classification_report(test_Y, y_pred))
    print(accuracy_score(test_Y, y_pred))
prediction_evaluation("SVM",classifier_rbf,X_test,y_test)

[[93 50  2 22]
 [46 98 15 37]
 [17 29 25  8]
 [21 56  3 31]]
              precision    recall  f1-score   support

           1       0.53      0.56      0.54       167
           2       0.42      0.50      0.46       196
           3       0.56      0.32      0.40        79
           4       0.32      0.28      0.30       111

   micro avg       0.45      0.45      0.45       553
   macro avg       0.45      0.41      0.42       553
weighted avg       0.45      0.45      0.44       553

0.44665461121157324


In [123]:
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB()
clf_NB.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [124]:
prediction_evaluation("MultinomialNB",clf_NB,X_test,y_test)

[[ 24 143   0   0]
 [  7 189   0   0]
 [  5  74   0   0]
 [  5 106   0   0]]
              precision    recall  f1-score   support

           1       0.59      0.14      0.23       167
           2       0.37      0.96      0.53       196
           3       0.00      0.00      0.00        79
           4       0.00      0.00      0.00       111

   micro avg       0.39      0.39      0.39       553
   macro avg       0.24      0.28      0.19       553
weighted avg       0.31      0.39      0.26       553

0.38517179023508136


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [125]:
from sklearn.neural_network import MLPClassifier
clf_mlp=MLPClassifier(hidden_layer_sizes=(500,200,), activation='relu', solver='adam', max_iter=1000,alpha=0.001)
clf_mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 200), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [126]:
prediction_evaluation("MLPClassifier",clf_mlp,X_test,y_test)

[[ 83  51   4  29]
 [ 43 110  11  32]
 [ 15  33  24   7]
 [ 24  63   1  23]]
              precision    recall  f1-score   support

           1       0.50      0.50      0.50       167
           2       0.43      0.56      0.49       196
           3       0.60      0.30      0.40        79
           4       0.25      0.21      0.23       111

   micro avg       0.43      0.43      0.43       553
   macro avg       0.45      0.39      0.40       553
weighted avg       0.44      0.43      0.43       553

0.43399638336347196


In [127]:
from sklearn.linear_model import LogisticRegressionCV
clf_lg=LogisticRegressionCV(max_iter=5000,multi_class="multinomial")
clf_lg.fit(X_train,y_train)
prediction_evaluation("logistic",clf_lg,X_test,y_test)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[[ 87  59   2  19]
 [ 45 119   7  25]
 [ 17  36  20   6]
 [ 21  65   1  24]]
              precision    recall  f1-score   support

           1       0.51      0.52      0.52       167
           2       0.43      0.61      0.50       196
           3       0.67      0.25      0.37        79
           4       0.32      0.22      0.26       111

   micro avg       0.45      0.45      0.45       553
   macro avg       0.48      0.40      0.41       553
weighted avg       0.47      0.45      0.44       553

0.45207956600361665


In [60]:
data_k="now i cant live without you,i'm nothing without you.now i cant live without you, nothing without you.if i go away from you, be away from myself.because its only you,now its only you,you are my life now,peace in me and the pain too,now my love is just you. whats the relation between you and me!that i cant get away for a moment... i live for you everyday,i give every moment of my life to you.there's no moment without you,there's your name in every breath. because its only you,now its only you,you are my life now,peace in me and the pain too,You are my love,cause its only you,because its only you,now my love is just you"

In [30]:
data_k=tfidf.fit_transform([data_k])

In [33]:
print(data_k)

  (0, 8)	0.2750095491084634
  (0, 10)	0.09166984970282113
  (0, 12)	0.18333969940564226
  (0, 0)	0.2750095491084634
  (0, 1)	0.2750095491084634
  (0, 13)	0.5500190982169268
  (0, 7)	0.2750095491084634
  (0, 15)	0.18333969940564226
  (0, 14)	0.18333969940564226
  (0, 9)	0.2750095491084634
  (0, 6)	0.18333969940564226
  (0, 16)	0.09166984970282113
  (0, 11)	0.2750095491084634
  (0, 5)	0.09166984970282113
  (0, 4)	0.18333969940564226
  (0, 17)	0.18333969940564226
  (0, 2)	0.09166984970282113
  (0, 3)	0.09166984970282113


In [116]:
clf_lg.predict(data_k)

ValueError: X has 18 features per sample; expecting 6145

In [117]:
X_test[0]

<1x6145 sparse matrix of type '<class 'numpy.float64'>'
	with 69 stored elements in Compressed Sparse Row format>

In [61]:
data_m = df['lyrics'].values
data_m=np.append(data_m,data_k)

In [62]:
z=tfidf.fit_transform(data_m)

In [63]:
print(z.shape)

(778, 6145)


In [66]:
classifier_rbf.predict(z[777])

array(['4'], dtype=object)